In [1]:
!pip install spotipy

In [7]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
import spotipy.util as util
import spotipy.oauth2 as oauth2
import os
import datetime
import json
date_time = datetime.datetime.today() - datetime.timedelta(days=7)

In [8]:
username = "mr_q_5" # INSERT YOUR USERNAME HERE

os.environ["SPOTIPY_CLIENT_ID"] = '67bafffe4ec743408a81a7ceb10106b5' # client id
os.environ["SPOTIPY_CLIENT_SECRET"] = 'ac001e1fac7944e5a786637484adb5d1' # Secret ID
os.environ["SPOTIPY_REDIRECT_URI"] = 'http://localhost:7777/callback' # Redirect URI

In [10]:
client_credentials_manager = SpotifyClientCredentials(client_id="SPOTIPY_CLIENT_ID",client_secret="SPOTIPY_CLIENT_SECRET")
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
# oauth2.refresh_access_token()
scope = 'playlist-modify-private playlist-modify-public user-follow-read user-library-read' # scope needed for your programme 
token = util.prompt_for_user_token(username, scope)
if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)


In [11]:
def processAlbum(newAlbums, results, spotify):
    albums = results['items']
    while results['next']:
        results = spotify.next(results)
        albums.extend(results['items'])

    for album in albums:
        if (album['release_date'] > str(date_time)):
            newAlbums.append(album['id'])
    return newAlbums

In [12]:
def checkIfNewAlbums(artistId):
    artist = 'spotify:artist:' + artistId
    spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

    newAlbums = []
    
    results = spotify.artist_albums(artist, album_type='album')
    newAlbums = processAlbum(newAlbums, results, spotify)
    
    results = spotify.artist_albums(artist, album_type='single')
    newAlbums = processAlbum(newAlbums, results, spotify)
    
    return newAlbums


In [13]:
def generateFollowingArtistIdList():
    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

    results = sp.current_user_followed_artists(limit=50)
    resultString = json.dumps(results)
    split = resultString.split('https://open.spotify.com/artist/')
    followedArtistsId = []


    for i in range (1, len(split)):
        id = split[i].split('\"')
        followedArtistsId.append(id[0])

    while len(split) > 1: 
        results = sp.current_user_followed_artists(limit=50, after=followedArtistsId[len(followedArtistsId) - 1])
        resultString = json.dumps(results)
        split = resultString.split('https://open.spotify.com/artist/')
        for i in range (1, len(split)):
            id = split[i].split('\"')
            followedArtistsId.append(id[0])
    return followedArtistsId


In [14]:
def createNewPlaylist():
    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))
    user_id = sp.me()['id']
    playlistId = sp.user_playlist_create(user_id, 'New Songs for the Week')
    return playlistId

In [15]:
def makeSurePlaylistExists():
    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))
    user_id = sp.me()['id']
    allPlaylists = []

    playlistId = None
    playlists = sp.user_playlists(username, limit = 50)

    for playlist in playlists['items']:
        allPlaylists.append(playlist['name'])
        if playlist['name'] == 'New Songs for the Week':
            playlistId = playlist

    while(len(playlists['items']) != 0 ):
        playlists = sp.user_playlists(username, limit = 50, offset = len(allPlaylists))

        for playlist in playlists['items']:
            allPlaylists.append(playlist['name'])
            if (playlist['name'] == 'New Songs for the Week'):
                playlistId = playlist
    
    
    if (playlistId != None):
        sp.current_user_unfollow_playlist(playlistId['id'])
    playlistId = createNewPlaylist()
        
    return playlistId

In [16]:
def fillPlaylist(playlistId, albumId, artistId):
    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))
    songArray = []

    results = sp.album_tracks(albumId)
    songs = results['items']

    while results['next']:
        results = sp.next(results)
        songs.extend(results['items'])

    for song in songs:
        songString = json.dumps(song)
        if str(artistId) in songString:
            songArray.append(song['id'])
    sp.playlist_add_items(playlistId, songArray)

In [17]:
playlist = makeSurePlaylistExists()
playlistId = playlist["id"]

artistId = '2AfU5LYBVCiCtuCCfM7uVX'

followedArtistsId = generateFollowingArtistIdList()
for artistId in followedArtistsId:
    albumId = checkIfNewAlbums(artistId)
    # print(albumId)
    for album in albumId:
        fillPlaylist(playlistId, album, artistId)
print("Done")

TypeError: 'NoneType' object is not subscriptable